# Aufgabe-C-Sturm

## Allgemeines

Eine allgemeine Beschreibung der Laboraufgaben inklusive des Vorgehens, den Bewertungsrichtlinien und der Abgabe finden Sie  <a href="ML-allgemein.ipynb">hier</a>

## Datenquelle


* Laden Sie ihre Daten von http://141.72.190.207/ml_lab/C_sturm herunter
    * Die Daten sind geschützt. 
        * Sie müssen evtl. in einem Netzwerk der DHBW (z.B. WLAN, VPN, ...) angemeldet sein. 
        * Sie können sich auf der Webseite mit dem Benutzernamen dhbw und dem Zugangsnamen "ml_LaB_4$" anmelden. 
* Die Daten sind in einem anwendungsspezifischen Format gespeichert.
    * Sie finden evtl. Informationen über die Daten in einer "README" Datei. 
    * Finden Sie keine solche Datei sind die Daten selbst erklärend. 
    
    



## Aufgabe

Sagen Sie die Windgeschwindigkeit eines Sturms (in Knoten) aufgrund von Satellitenfotos vorher.
Der Datensatz besteht aus Fotos von 494 verschiedenen Stürmen im Atlantik und Pazifik mit ihren zugehörigen Windgeschwindigkeiten.
Jedes Bild hat `366 x 366` Pixel, und es sind 70.257 Trainingsdaten und 45.377 Testdaten vorhanden.
Die Bilder wurden zu mehreren Zeitpunkten während der Lebensdauer eines Sturms aufgenommen.

Für jeden Sturm im Trainings und Testdatensatz erhalten Sie eine Zeitreihe von Bildern mit der jeweiligen assoziierten relativen Zeit seit Beginn des Sturms.
Ihr Modell sollte neben den reinen Bilddaten also auch den zeitlichen Verlauf des Sturms betrachten, um die Vorhersage für künftige Vorhersagezeitpunkte zu erstellen

Die Bilder sind nach folgendem Schema benannt: `{image_id}.jpg`.
Diese IDs bestehen aus einer Sturm_ID und einer Bildnummer entsprechend der zeitlichen Bildreihenfolge.

Ihr Ziel ist es, für die Testdaten die korrekte Windgeschwindigkeiten vorherzusagen.

# Lösung

* Beginnen Sie hier mit Ihrer Dokumentation und Implementierung! 

Geplantes Vorgehen: 

- Daten vorbereitung
- CNN-Modell trainieren (für räumliche Merkmale der Bilder)
- LSTM-Modell trainieren (für zeitlichen Verlauf der Bilder)
- Kombinierung der Modelle (Fully Connected-Layer um CNN in LSTM einzuspeisen, Ausgabe von LSTM ist finale Vorhersage)
- Kombinierte Modell auf Trainingsdaten trainieren 

In [1]:
import os
import json
import numpy as np
from PIL import Image

# Set the paths to the data directories
train_data_dir = 'storm_train_data'
train_labels_dir = 'storm_train_labels'
test_data_dir = 'storm_test_data'
test_labels_dir = 'storm_test_labels'

# Define the image size and number of channels
img_size = (366, 366)
num_channels = 1

# Define the sequence length
seq_length = 10

# Function to load and preprocess a single image
def load_image(file_path):
    img = Image.open(file_path).convert('L') # Open the image and convert to grayscale
    img = img.resize(img_size) # Resize the image
    img_array = np.array(img) # Convert to numpy array
    img_array = img_array.reshape((*img_size, num_channels)) # Add channel dimension
    img_array = img_array / 255.0 # Normalize pixel values to [0, 1]
    return img_array

# Function to load and preprocess the data for a single storm
def load_storm_data(storm_dir):
    # Load the features for this storm
    with open(os.path.join(train_data_dir, storm_dir, 'features.json'), 'r') as f:
        features = json.load(f)
    # Load the labels for this storm
    with open(os.path.join(train_labels_dir, storm_dir, 'labels.json'), 'r') as f:
        labels = json.load(f)
    # Load and preprocess the images for this storm
    img_sequences = []
    label_sequences = []
    for i, img_id in enumerate(features.keys()):
        # Load the image
        img = load_image(os.path.join(train_data_dir, storm_dir, 'image.jpg'))
        # Add the image to the sequence
        if i >= seq_length - 1:
            img_seq = [load_image(os.path.join(train_data_dir, storm_dir, f'image_{j}.jpg')) 
                       for j in range(i-seq_length+1, i+1)]
            img_sequences.append(np.concatenate(img_seq, axis=-1))
            label_sequences.append(labels[img_id]['wind_speed'])
    return np.array(img_sequences), np.array(label_sequences)

# Function to load and preprocess the data for all storms
def load_data():
    # Load and preprocess the training data
    train_img_sequences = []
    train_label_sequences = []
    for storm_dir in os.listdir(train_data_dir):
        if os.path.isdir(os.path.join(train_data_dir, storm_dir)):
            img_sequences, label_sequences = load_storm_data(storm_dir)
            train_img_sequences.append(img_sequences)
            train_label_sequences.append(label_sequences)
    train_img_sequences = np.concatenate(train_img_sequences, axis=0)
    train_label_sequences = np.concatenate(train_label_sequences, axis=0)
    # Load and preprocess the test data
    test_img_sequences = []
    test_label_sequences = []
    for storm_dir in os.listdir(test_data_dir):
        if os.path.isdir(os.path.join(test_data_dir, storm_dir)):
            img_sequences, label_sequences = load_storm_data(storm_dir)
            test_img_sequences.append(img_sequences)
            test_label_sequences.append(label_sequences)
    test_img_sequences = np.concatenate(test_img_sequences, axis=0)
    test_label_sequences = np.concatenate(test_label_sequences, axis=0)
    return train_img_sequences, train_label_sequences, test_img_sequences, test_label_sequences

# Load the data
train_data, train_labels, test_data, test_labels = load_data()


FileNotFoundError: [Errno 2] No such file or directory: 'storm_train_data\\acd_123\\features.json'